In [4]:
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import multivariate_normal as mvnorm
import corner
import copy
import multiprocessing as mp
from tqdm import tqdm
from functools import partial

from hmc import HMC
from distributions import UnivariateNormal, IndependentMultivariateNormal, MultivariateNormal

In [5]:
# Generate observed data
truth = (0,1)

N = 10000
data = norm.rvs(loc=truth[0], scale=truth[1], size=N)

print(f'mu_hat: {np.mean(data)}, sig_hat: {np.std(data)}')

mu_hat: 0.011061193339785785, sig_hat: 0.98866663692596


In [9]:
# Initialize Sampler
dist = UnivariateNormal()
# Initialize Sampler
sampler = HMC(dist.logp, dist.dlogp, dt=1e-4, L=10, M=0.1*np.identity(2), n_args=2)
# Run sampling
N_chains = mp.cpu_count()-1
# Run sampling
xs = sampler.sample(int(1e4), data, init_x=[0.1,0.5], verbose=True)

  7%|▋         | 734/9999 [00:02<00:28, 328.99it/s]

KeyboardInterrupt: 

In [10]:
# Generate observed data
ndims = 5 # Number of dimensions for multivariate Gaussian

mu_tru = 10*np.random.rand(ndims)
sig_tru = 5*np.random.rand(ndims)

N = 10000
data = mvnorm.rvs(mean=mu_tru, cov=np.diag(sig_tru), size=N)

print(f'mu_*: {mu_tru}, sig_*: {np.diag(sig_tru)}')
print(f'mu_hat: {np.mean(data)}, sig_hat: {np.cov(data)}')

mu_*: [3.3845899  3.95632212 3.30771196 1.92781996 3.59425344], sig_*: [[3.80808876 0.         0.         0.         0.        ]
 [0.         3.26279971 0.         0.         0.        ]
 [0.         0.         3.15200966 0.         0.        ]
 [0.         0.         0.         3.8305056  0.        ]
 [0.         0.         0.         0.         3.64892337]]
mu_hat: 3.2386788217337017, sig_hat: [[ 3.71950815  1.90386355  0.36138862 ...  1.34070713  0.89621175
   0.88193599]
 [ 1.90386355  4.05525123  2.82417004 ...  3.75890187  0.62836644
   0.06984554]
 [ 0.36138862  2.82417004  2.345598   ...  2.74321665  0.14841794
  -0.78011278]
 ...
 [ 1.34070713  3.75890187  2.74321665 ...  3.57011786  0.69857817
   0.02050792]
 [ 0.89621175  0.62836644  0.14841794 ...  0.69857817  2.41034582
  -0.38610767]
 [ 0.88193599  0.06984554 -0.78011278 ...  0.02050792 -0.38610767
   7.20425159]]


  7%|▋         | 734/9999 [00:20<00:28, 328.99it/s]

In [ ]:

# Initialize Sampler
sampler = HMC(logp, dlogp, dt=1e-4, L=10, M=0.1*np.identity(2*ndims), n_args=2*ndims)
# Run sampling
N_chains = mp.cpu_count()-1
xs = sampler.sample_pool(int(1e4), data, N_chains, 
                         init_x = np.concatenate([mu_tru, sig_tru]), verbose=True)